In [2]:
import numpy as np
np.set_printoptions(precision=3, suppress=True)
import scipy.sparse as sp
from scipy.stats import norm
from graphite_maps.precision_estimation import fit_precision_cholesky, precision_to_graph

Set up a simple AR-1 example to test `graphite_maps` functionality.
$$u_t = \phi u_{t-1} + \epsilon,~~
 \epsilon\sim N(0,1) $$

In [3]:
# Define parameters
n = 1000  # Sample size
p = 1000   # Dimension
phi = 0.6

In [4]:
# Simulate data
def rar1(T, phi):
    """simulate auto-regressive-1.
    The first element is simulated from stationary distribution.
    """
    x = np.empty([T])
    x[0] = np.random.normal(0, 1 / np.sqrt(1 - phi**2))
    for i in range(1, T):
        x[i] = phi * x[i - 1] + np.random.normal(0, 1)
    return x

np.random.seed(42)
U = np.array([rar1(T=p, phi=phi) for _ in range(n)])

We need the graph of conditional independence in $u$, 
thus create precision (which is conceptually unknown) and find graph from this (conceptually already known).

In [5]:
# create AR-1 precision matrix
prec = sp.diags(
    [np.repeat(-phi, p-1), np.concatenate(([1.0], np.repeat(1.0+phi**2, p-2), [1.0])), np.repeat(-phi, p-1)], 
    [-1,0,1], shape=(p, p), 
    format='csc'
)

Graph_u = precision_to_graph(prec)

Now use `graphite_maps` to estimate the precision w.r.t. $G_u$

In [6]:
prec_est = fit_precision_cholesky(U, Graph_u, verbose_level=2)

Parameters in precision: 1999
Parameters in Cholesky factor: 1999
Starting statistical fitting of precision


Learning precision Cholesky factor row-by-row: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 486.88it/s]

Precision has log-determinant: 0.8304692396087653


Inspect that the result seem to converge (no values should be too large here).

In [7]:
print(prec_est.A - prec.A)
print(np.max(np.abs(prec_est-prec)))

[[-0.005 -0.012  0.    ...  0.     0.     0.   ]
 [-0.012 -0.028  0.096 ...  0.     0.     0.   ]
 [ 0.     0.096 -0.162 ...  0.     0.     0.   ]
 ...
 [ 0.     0.     0.    ...  0.012  0.012  0.   ]
 [ 0.     0.     0.    ...  0.012 -0.023 -0.006]
 [ 0.     0.     0.    ...  0.    -0.006  0.035]]
0.2178122656748398
